[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/%E7%BB%8F%E5%85%B8%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C_AlexNet.ipynb)

本章将介绍4种经典的卷积神经网络，分别是AlexNet、VGGNet、Google Inception Net、ResNet，这4中网络依照出现的先后顺序排列，深度和复杂度也依次递进。

## AlexNet
AlexNet主要使用到的新技术点如下：
- 成功使用ReLU作为CNN的激活函数，并验证其效果在较深的网络超过了sigmoid，成功解决了sigmoid在网络较深时的梯度弥散问题。
- 训练时使用Dropout随机忽略一部分神经元，以避免模型过拟合。
- 在CNN中使用重叠的最大池化。此前CNN中普遍使用平均池化，AlexNet全部使用最大池化，避免平均池化的模糊化效果。并且AlexNet中提出让步长比池化和的尺寸小，这样池化层的输出之间会有重叠和覆盖，提升了特征的丰富性
- 提出了LRN层，对局部神经元的活动创建竞争机制，使得其中响应比较大的值变得相对更大，并抑制其他反馈较小的神经元，增强了模型的泛化能力
- 使用CUDA加速深度卷积网络的训练，利用GPU强大的并行计算能力，处理神经网络训练时大量的矩阵运算。
- 数据增强，随机的从256x256的原始图像中街区224x224大小的区域(以及水平翻转的镜像)，相当于增加了$(256-224)^2x2=2048$倍的数据量。防止过拟合


In [0]:
from datetime import datetime
import math
import time
import tensorflow as tf

In [0]:
batch_size = 32
num_batches=100

In [0]:
#显示网络每一层结构的函数print_actications，展示每一个卷积层或池化层输出tensor的尺寸
def print_activations(t):
  print(t.op.name, ' ', t.get_shape().as_list())

### 设计AlexNet的网络结构

In [9]:
'''
#接受images作为输入，返回最后一层pool5(第5个池化层)及parameters(AlexNet中所有需要训练的模型参数)

def inference(images):
  parameters = []
  
  with tf.name_scope('conv1') as scope:
    kernel = tf.Variable(tf.truncated_normal([11,11,3,64],
                                            dtype=tf.float32, stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(images, kernel,[1,4,4,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[64],dtype=tf.float32),trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope)# 使用relu激活函数对结果进行非线性处理
    print_activations(conv1)#输出卷积层的结构
    parameters += [kernel, biases] #把这一层可训练的参数添加到parameters中
    
    #在第一个卷积层后再添加LRN层和最大池化层
    lrn1 = tf.nn.lrn(conv, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn1')#对前面输出的tensor conv1进行LRN处理
    pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1],padding='VALID',name='pool1')#最大池化处理，padding为VALID，即取样时不能超过边框，不想SAME模式那样可以填充边界外的点
    print_activations(pool1)#可视化池化结果
    
    
  #设计第二个卷积层  
  with tf.name_scope('conv2') as scope:
    kernel = tf.Variable(tf.truncated_normal([5,5,64,192],
                                            dtype=tf.float32, stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(pool1, kernel, [1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[192],
                                    dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope)
    parameters += [kernel, biases]
    print_activations(conv2)
  
    #对第2个卷积层的输出conv2进行处理，同样是先做LRN处理，在进行最大池化处理，参数和之前完全一样
    lrn2 = tf.nn.lrn(conv2, 4,bias=1.0,alpha=0.001/9, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1,3,3,1], strides=[1,2,2,1],
                        padding='VALID', name='pool2')
    print_activations(pool2)
  
  #创建第三层卷积层
  with tf.name_scope('conv3') as scope:
    kernel = tf.Variable(tf.truncated_normal([3,3,192,384],
                                            dtype=tf.float32, stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(pool2, kernel,[1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[384],
                                    dtype=tf.float32),trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv3 = tf.nn.relu(bias, name=scope)
    parameters += [kernel, biases]
    print_activations(conv3)
  
  #第四个卷积层
  with tf.name_scope('conv4') as scope:
    kernel = tf.Variable(tf.truncated_normal([3,3,384,256],
                                            dtype=tf.float32, stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(conv3, kernel, [1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[256],
                                    dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv4 = tf.nn.relu(bias, name=scope)
    parameters += [kernel, biases]
    print_activations(conv4)
    
  #第五个卷积层
  with tf.name_scope('conv5') as scope:
    kernel = tf.Variable(tf.truncated_normal([3,3,256,256],
                        dtype=tf.float32, stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(conv4, kernel, [1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv5 = tf.nn.relu(bias, name=scope)
    parameters += [kernel, biases]
    print_activations(conv5)
    
    pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool5')
    print_activations(pool5)
  
  return pool5,parameters
  '''

"\n#接受images作为输入，返回最后一层pool5(第5个池化层)及parameters(AlexNet中所有需要训练的模型参数)\n\ndef inference(images):\n  parameters = []\n  \n  with tf.name_scope('conv1') as scope:\n    kernel = tf.Variable(tf.truncated_normal([11,11,3,64],\n                                            dtype=tf.float32, stddev=1e-1), name='weights')\n    conv = tf.nn.conv2d(images, kernel,[1,4,4,1], padding='SAME')\n    biases = tf.Variable(tf.constant(0.0, shape=[64],dtype=tf.float32),trainable=True, name='biases')\n    bias = tf.nn.bias_add(conv, biases)\n    conv1 = tf.nn.relu(bias, name=scope)# 使用relu激活函数对结果进行非线性处理\n    print_activations(conv1)#输出卷积层的结构\n    parameters += [kernel, biases] #把这一层可训练的参数添加到parameters中\n    \n    #在第一个卷积层后再添加LRN层和最大池化层\n    lrn1 = tf.nn.lrn(conv, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn1')#对前面输出的tensor conv1进行LRN处理\n    pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1],padding='VALID',name='pool1')#最大池化处理，padding为VALID，即取样时不能超过边框，不想SAME模式那样可以填充边界外的点\n    print_activation

In [0]:
def inference(images):
    parameters = []
    
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11,11,3,64],
                                                 stddev=1e-1, dtype=tf.float32,name='weights'))
        conv = tf.nn.conv2d(images,kernel,[1,4,4,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[64], dtype=tf.float32, 
                                           name='biaes'))
        
        bias = tf.nn.bias_add(conv,biases)
        
        conv1 = tf.nn.relu(bias, name=scope)
        print_activations(conv1)
        parameters += [kernel,biases]
        
        
        lrn1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn1')
        pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1],
                                padding='VALID', name='pool1')
        
        print_activations(pool1)
        
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5,5,64,192],
                                                 stddev=1e-1, dtype=tf.float32,name='weights'))
        conv = tf.nn.conv2d(pool1,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0, dtype=tf.float32, shape=[192], name='biaes'))
        
        bias = tf.nn.bias_add(conv,biases)
        
        conv2 = tf.nn.relu(bias, name=scope)
        
        parameters += [kernel,biases]
        print_activations(conv2)    
            
        lrn2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn2')
        pool2 = tf.nn.max_pool(lrn2, ksize=[1,3,3,1], strides=[1,2,2,1],
                                padding='VALID', name='pool2')
        
        print_activations(pool2)
      

    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,192,384],
                                                 stddev=1e-1, dtype=tf.float32,name='weights'))
        conv = tf.nn.conv2d(pool2,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0, dtype=tf.float32, shape=[384], name='biaes'))
        
        bias = tf.nn.bias_add(conv,biases)
        
        conv3 = tf.nn.relu(bias, name=scope)
        
        parameters += [kernel,biases]
        print_activations(conv3)    
            
    
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,384,256],
                                                 stddev=1e-1, dtype=tf.float32,name='weights'))
        conv = tf.nn.conv2d(conv3,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0, dtype=tf.float32, shape=[256],
                                          name='biaes'))
        
        bias = tf.nn.bias_add(conv,biases)
        
        conv4 = tf.nn.relu(bias, name=scope)
        
        parameters += [kernel,biases]
        print_activations(conv4)    
            
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,256,256],
                                                 stddev=1e-1, dtype=tf.float32,name='weights'))
        conv = tf.nn.conv2d(conv4,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0, dtype=tf.float32, shape=[256],
                                          name='biaes'))
        
        bias = tf.nn.bias_add(conv,biases)
        
        conv5 = tf.nn.relu(bias, name=scope)
        
        parameters += [kernel,biases]
        print_activations(conv5)   
        
        pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,2,2,1],
                                    padding='VALID', name='pool5')
            
        print_activations(pool5)
        
    return pool5,parameters

In [0]:
def time_tensorflow_run(session,target,info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s:step %d,duration = %.3f' %(datetime.now(),i - num_steps_burn_in,duration))
            
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s:%s across %d steps,%.3f +/- %.3f sec / batch'%
          (datetime.now(),info_string,num_batches,mn,sd))
    
def run_benchmark():
  #主函数run_benchmark
    with tf.Graph().as_default():
        image_size = 224
        #使用random_normal函数构造正态分布(标准差为0.1)的随机tensor
        images = tf.Variable(tf.random_normal([batch_size,
                                               image_size,
                                               image_size,3],
                                               stddev=1e-1,
                                               dtype=tf.float32))
        pool5,parameters = inference(images)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        time_tensorflow_run(sess, pool5, "Forword")
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective,parameters)
        time_tensorflow_run(sess, grad, "Forword-backward")

In [8]:
run_benchmark()

conv1   [32, 56, 56, 64]
conv1/pool1   [32, 27, 27, 64]
conv2   [32, 27, 27, 192]
conv2/pool2   [32, 13, 13, 192]
conv3   [32, 13, 13, 384]
conv4   [32, 13, 13, 256]
conv5   [32, 13, 13, 256]
conv5/pool5   [32, 6, 6, 256]
2018-05-03 10:14:31.962916:step 0,duration = 0.049
2018-05-03 10:14:32.441134:step 10,duration = 0.047
2018-05-03 10:14:32.915056:step 20,duration = 0.047
2018-05-03 10:14:33.390633:step 30,duration = 0.048
2018-05-03 10:14:33.865000:step 40,duration = 0.047
2018-05-03 10:14:34.339283:step 50,duration = 0.047
2018-05-03 10:14:34.813369:step 60,duration = 0.047
2018-05-03 10:14:35.287235:step 70,duration = 0.048
2018-05-03 10:14:35.760969:step 80,duration = 0.047
2018-05-03 10:14:36.235670:step 90,duration = 0.047
2018-05-03 10:14:36.662592:Forword across 100 steps,0.047 +/- 0.000 sec / batch
2018-05-03 10:14:38.462426:step 0,duration = 0.134
2018-05-03 10:14:39.813483:step 10,duration = 0.136
2018-05-03 10:14:41.165236:step 20,duration = 0.135
2018-05-03 10:14:42.5149